In [2]:
import pandas as pd
from pathlib import Path
import sys
import os
# Ajoute le répertoire parent au chemin de recherche de Python
sys.path.append(os.path.join(str(Path().resolve().parent),"gestionbdd"))
import getdata


In [ ]:
debut = "2023-01-01"
fin   = "2023-12-31"
# Chargement des données stockées dans PostGres

data_occup_jour  = getdata.get_data(source = "occup_jour")
data_elec_jour   = getdata.get_data(source = "gaz_jour")
data_gaz_jour    = getdata.get_data(source = "gaz_jour")
data_meteo_jour  = getdata.get_data(source = "meteo_jour")

data_occup_jour  = data_occup_jour [(data_occup_jour .index>=debut) & (data_occup_jour.index<=fin)]
data_elec_jour   = data_elec_jour[(data_elec_jour.index>=debut) & (data_elec_jour.index<=fin)]
data_gaz_jour    = data_gaz_jour[(data_gaz_jour.index>=debut) & (data_gaz_jour.index<=fin)]
data_meteo_jour  = data_meteo_jour[(data_meteo_jour.index>=debut) & (data_meteo_jour.index<=fin)]
data_occup_jour['presence'] = data_occup_jour['presence'].apply(lambda o: 1.0 if o=="oui" else 0.0)
T_moyenne = (data_meteo_jour['temperature_2m_min'] + data_meteo_jour['temperature_2m_max']) / 2
data_meteo_jour = data_meteo_jour.assign(DJU=lambda x: (18.0 - T_moyenne).clip(lower=0))
data_model_elec  = pd.concat([data_elec_jour['value'],data_meteo_jour['DJU'],data_occup_jour['presence']], axis=1)

source de données inconnue


## Modèle jours

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
x = data_model_elec[['DJU','presence']]
y = data_model_elec['value']
model.fit(X=x,y=y)
from sklearn.metrics import r2_score
r2_score(y,model.predict(x) )

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [36]:
from sklearn.metrics import r2_score
r2_score(y,model.predict(x) )

0.6244836854473995

In [37]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=x.index,
    y=y,
    mode='lines',
    name="mesure"  # nom affiché dans la légende
))


fig.add_trace(go.Scatter(
    x=x.index,
    y=model.predict(x),
    mode='lines',
    name="modele"  # nom affiché dans la légende
))

fig.update_layout(
    title="Évolution des colonnes temporelles",
    xaxis_title="Date",
    yaxis_title="Valeurs",
    hovermode="x unified"
)

fig.show()

## Modèle Mois Elec

In [121]:
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy import stats
from joblib import dump

debut = "2022-01-01"
fin   = "2023-12-31"
data_occup_jour  = getdata.get_data(source = "occup_jour")
data_elec_jour   = getdata.get_data(source = "elec_jour")
data_gaz_jour    = getdata.get_data(source = "gaz_jour")
data_meteo_jour  = getdata.get_data(source = "meteo_jour")

data_occup_jour  = data_occup_jour [(data_occup_jour .index>=debut) & (data_occup_jour.index<=fin)]
data_elec_jour   = data_elec_jour[(data_elec_jour.index>=debut) & (data_elec_jour.index<=fin)]
data_meteo_jour  = data_meteo_jour[(data_meteo_jour.index>=debut) & (data_meteo_jour.index<=fin)]

data_occup_jour['presence'] = data_occup_jour['presence'].apply(lambda o: 1.0 if o=="oui" else 0.0)
T_moyenne = (data_meteo_jour['temperature_2m_min'] + data_meteo_jour['temperature_2m_max']) / 2
data_meteo_jour = data_meteo_jour.assign(DJU=lambda x: (18.0 - T_moyenne).clip(lower=0))

data_model  = pd.concat([data_elec_jour['value'],data_meteo_jour['DJU'],data_occup_jour['presence']], axis=1)


data_model_mois = data_model.resample('MS').sum()

data_model_mois['presence'] = 100*data_model_mois['presence']/data_model.resample("MS").size()

model = LinearRegression()
x = data_model_mois[['DJU','presence']]
y = data_model_mois['value']
model.fit(X=x,y=y)

print(r2_score(y,model.predict(x) ))


y_pred = model.predict(x)

# Calcul de la variance résiduelle
n, p = x.shape
residuals = y - y_pred
sigma2 = np.sum(residuals**2) / (n - p - 1)

# Matrice avec constante
X_design = np.column_stack([np.ones(n), x])
XtX_inv = np.linalg.inv(X_design.T @ X_design)

# Ajout d'une constante pour l'intercept
X_design = np.column_stack([np.ones(n), x])
# Nouveau point à prédire
x0 = np.array([1, 350, 95.0])  # <-- constante + valeurs
y0_pred = model.predict(pd.DataFrame([{"DJU": 350, "presence": 95.0}]))[0]

# Erreur standard de la prédiction
se_y0 = np.sqrt(sigma2 * (x0.T @ XtX_inv @ x0))

# Intervalle de confiance 95 %
t_val = stats.t.ppf(1 - 0.025, df=n - p - 1)
ci_lower = y0_pred - t_val * se_y0
ci_upper = y0_pred + t_val * se_y0

print(f"Prediction: {y0_pred:.3f}")
print(f"IC 95%: [{ci_lower:.3f}, {ci_upper:.3f}]")


frequence = "mois"
energie   = "elec"
model_name = f"model-{energie}-{frequence}.joblib"

model_config = {
    "model": model,
    "x_train": x,
    "t_val": t_val,
    "XtX_inv": XtX_inv,
    "sigma2 ":sigma2 
}

dump(model_config,"../ml_models/"+model_name)










source de données inconnue
0.8258757334221482
Prediction: 1428.316
IC 95%: [1325.325, 1531.307]


['../ml_models/model-elec-mois.joblib']

In [125]:
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy import stats
from joblib import dump

debut = "2022-11-01"
fin   = "2023-12-31"
data_occup_jour  = getdata.get_data(source = "occup_jour")
data_gaz_jour    = getdata.get_data(source = "gaz_jour")
data_meteo_jour  = getdata.get_data(source = "meteo_jour")

data_occup_jour  = data_occup_jour [(data_occup_jour .index>=debut) & (data_occup_jour.index<=fin)]
data_meteo_jour  = data_meteo_jour[(data_meteo_jour.index>=debut) & (data_meteo_jour.index<=fin)]
data_gaz_jour    = data_gaz_jour[(data_gaz_jour.index>=debut) & (data_gaz_jour.index<=fin)]

data_occup_jour['presence'] = data_occup_jour['presence'].apply(lambda o: 1.0 if o=="oui" else 0.0)
T_moyenne = (data_meteo_jour['temperature_2m_min'] + data_meteo_jour['temperature_2m_max']) / 2
data_meteo_jour = data_meteo_jour.assign(DJU=lambda x: (18.0 - T_moyenne).clip(lower=0))

data_model  = pd.concat([data_gaz_jour['energie'],data_meteo_jour['DJU'],data_occup_jour['presence']], axis=1)


data_model_mois = data_model.resample('MS').sum()

data_model_mois['presence'] = 100*data_model_mois['presence']/data_model.resample("MS").size()

model = LinearRegression()
x = data_model_mois[['DJU','presence']]
y = data_model_mois['energie']
model.fit(X=x,y=y)

print(r2_score(y,model.predict(x) ))


y_pred = model.predict(x)

# Calcul de la variance résiduelle
n, p = x.shape
residuals = y - y_pred
sigma2 = np.sum(residuals**2) / (n - p - 1)

# Matrice avec constante
X_design = np.column_stack([np.ones(n), x])
XtX_inv = np.linalg.inv(X_design.T @ X_design)

# Ajout d'une constante pour l'intercept
X_design = np.column_stack([np.ones(n), x])
# Nouveau point à prédire
x0 = np.array([1, 350, 95.0])  # <-- constante + valeurs
y0_pred = model.predict(pd.DataFrame([{"DJU": 350, "presence": 95.0}]))[0]

# Erreur standard de la prédiction
se_y0 = np.sqrt(sigma2 * (x0.T @ XtX_inv @ x0))

# Intervalle de confiance 95 %
t_val = stats.t.ppf(1 - 0.025, df=n - p - 1)
ci_lower = y0_pred - t_val * se_y0
ci_upper = y0_pred + t_val * se_y0

print(f"Prediction: {y0_pred:.3f}")
print(f"IC 95%: [{ci_lower:.3f}, {ci_upper:.3f}]")


frequence = "mois"
energie   = "gaz"
model_name = f"model-{energie}-{frequence}.joblib"

model_config = {
    "model": model,
    "x_train": x,
    "t_val": t_val,
    "XtX_inv": XtX_inv,
    "sigma2 ":sigma2 
}

dump(model_config,"../ml_models/"+model_name)

source de données inconnue
0.9328661570446269
Prediction: 3012.512
IC 95%: [2624.872, 3400.153]


['../ml_models/model-gaz-mois.joblib']

In [3]:
from joblib import load
model_obj = load("../ml_models/model-elec-mois.joblib")

In [4]:
import getdata
#from modelisation.predict import predict_conso
from joblib import load


# Chargement des données stockées dans PostGres


data_elec_heure = getdata.get_data(source = "elec_hour")
data_elec_jour  = getdata.get_data(source = "elec_jour")
data_gaz_jour   = getdata.get_data(source = "gaz_jour")
data_meteo_jour = getdata.get_data(source = "meteo_jour")
data_occup_jour = getdata.get_data(source = "occup_jour")

data_elec_mois  = data_elec_jour['value'].resample("MS").sum()
data_gaz_mois   = data_gaz_jour['energie'].resample("MS").sum()

data_occup_jour['presence'] = data_occup_jour['presence'].apply(lambda o: 1.0 if o=="oui" else 0.0)


T_moyenne = (data_meteo_jour['temperature_2m_min'] + data_meteo_jour['temperature_2m_max']) / 2
data_meteo_jour = data_meteo_jour.assign(DJU=lambda x: (18.0 - T_moyenne).clip(lower=0))

data_occup_mois             = 100*data_occup_jour['presence'].resample("MS").sum()/data_occup_jour.resample("MS").size()
data_dju_mois = data_meteo_jour['DJU'].resample('MS').sum()

In [29]:
data_occup_mois.name = 'presence'
data_model_mois = pd.concat([data_elec_mois,data_dju_mois,data_occup_mois], axis=1)
data_model_mois.dropna(inplace=True)
features_name = model_obj['x_train'].columns
df_X = data_model_mois[features_name]


date_debut = model_obj['x_train'].index[0]
df_X = df_X[df_X.index>=date_debut]
y_pred = model_obj['model'].predict(df_X)
modele_elec_mois = pd.Series(index=df_X.index,data = y0_pred)

In [38]:
model_obj.keys()

dict_keys(['model', 'x_train', 't_val', 'XtX_inv', 'sigma2 '])

In [14]:
sigma2 = model_obj['sigma2 ']
XtX_inv = model_obj['XtX_inv']
t_val = model_obj['t_val']

In [30]:
import numpy as np
# facteur d'incertitude pour chaque prédiction

X = df_X.values  # matrice des features
X_design = np.hstack([np.ones((n, 1)), X])

In [27]:
# Erreur standard de la prédiction
se_y0 = np.sqrt(sigma2 * (x0.T @ XtX_inv @ x0))

C:\Users\David\AppData\Local\Temp\ipykernel_15712\618556624.py:2: RuntimeWarning: invalid value encountered in sqrt
  se_y0 = np.sqrt(sigma2 * (x0.T @ XtX_inv @ x0))


In [31]:
pred_var = np.sum(X_design @ XtX_inv * X_design, axis=1)

In [33]:
interval = t_val * np.sqrt(sigma2 * (1 + pred_var))

In [34]:
interval

array([328.40313684, 323.73248857, 314.51201909, 313.04285396,
       317.60227681, 325.74549287, 373.40310071, 340.50119704,
       319.23959196, 318.97487665, 313.42923013, 327.5263272 ,
       323.72499003, 319.4345411 , 315.43581084, 313.72583454,
       315.49323666, 329.48623161, 321.05075836, 335.19914001,
       326.94143429, 316.56386253, 314.20528079, 317.87472489,
       332.53533866, 314.12466397, 314.71724823, 312.85977572,
       315.72597336, 318.95013604, 335.07221617, 381.60766988,
       319.16257841, 314.51257408, 315.104113  , 322.91469061,
       333.541506  , 322.94908442, 316.742598  , 313.40669842,
       317.04810606, 326.74483755, 328.88344048, 345.82128293,
       317.60115673, 316.06395435])

In [ ]:

# Calcul de la variance résiduelle
n, p = x.shape
residuals = y - y_pred
sigma2 = np.sum(residuals**2) / (n - p - 1)

# Matrice avec constante
X_design = np.column_stack([np.ones(n), x])
XtX_inv = np.linalg.inv(X_design.T @ X_design)

# Ajout d'une constante pour l'intercept
X_design = np.column_stack([np.ones(n), x])
# Nouveau point à prédire
x0 = np.array([1, 350, 95.0])  # <-- constante + valeurs
y0_pred = model.predict(pd.DataFrame([{"DJU": 350, "presence": 95.0}]))[0]

# Erreur standard de la prédiction
se_y0 = np.sqrt(sigma2 * (x0.T @ XtX_inv @ x0))

# Intervalle de confiance 95 %
t_val = stats.t.ppf(1 - 0.025, df=n - p - 1)
ci_lower = y0_pred - t_val * se_y0
ci_upper = y0_pred + t_val * se_y0